In [1]:
import os
import warnings
import sys

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

In [3]:
import mlflow
import mlflow.sklearn

In [4]:
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [5]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [6]:
warnings.filterwarnings("ignore")
np.random.seed(40)

In [7]:
# Read the wine-quality csv file from the URL
csv_url =\
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
try:
    data = pd.read_csv(csv_url, sep=';')
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e)

In [8]:
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

In [9]:
alpha = 0.5
l1_ratio = 0.5

In [11]:
ep = 0
for i in np.arange(0,1,0.1):
    with mlflow.start_run():
        lr = ElasticNet(alpha=i, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (i, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", i)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse,step=ep)
        mlflow.log_metric("r2", r2,step=ep)
        mlflow.log_metric("mae", mae,step=ep)
        ep += 1

        mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.000000, l1_ratio=0.500000):
  RMSE: 0.6603811224461619
  MAE: 0.507364419988892
  R2: 0.3820931219110161
Elasticnet model (alpha=0.100000, l1_ratio=0.500000):
  RMSE: 0.7308996187375898
  MAE: 0.5615486628017713
  R2: 0.2430813606733676
Elasticnet model (alpha=0.200000, l1_ratio=0.500000):
  RMSE: 0.7415652207304311
  MAE: 0.573067857646195
  R2: 0.22082961765864062
Elasticnet model (alpha=0.300000, l1_ratio=0.500000):
  RMSE: 0.7573787958793152
  MAE: 0.5893143148791097
  R2: 0.18724431943947972
Elasticnet model (alpha=0.400000, l1_ratio=0.500000):
  RMSE: 0.7759342885655987
  MAE: 0.6090076377075831
  R2: 0.14693206734185604
Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614
Elasticnet model (alpha=0.600000, l1_ratio=0.500000):
  RMSE: 0.811295303072729
  MAE: 0.645693705089251
  R2: 0.06740807086129275
Elasticnet model (alpha=0.700000, l1_ratio=0.500000):
  RMSE: 0.82989218525

In [13]:
# mlflow.create_experiment('new_experiment')
mlflow.set_experiment('new')

INFO: 'new' does not exist. Creating a new experiment


In [15]:
mlflow.sklearn.save_model(lr,'ml_models/')

In [17]:
lr = mlflow.sklearn.load_model('mlruns/0/e4dd8a99dbce4a1ea55aa260cca8c684/artifacts/model/')